In [25]:
import pandas as pd
import numpy as np
import re
import warnings 
warnings.filterwarnings("ignore")

Создадим словарь названий для унификации субъектов РФ

In [26]:
names = {'адыгея':'Республика Адыгея (Адыгея)','башкорт':'Республика Башкортостан',
'бурятия':'Республика Бурятия','дагестан':'Республика Дагестан','ингуш':'Республика Ингушетия',
'кабардино':'Кабардино-Балкарская Республика', 'карачаево':'Карачаево-Черкесская Республика',
'крым':'Республика Крым', 'республика саха':'Республика Саха (Якутия)', 'северная':'Республика Северная Осетия — Алания',
'татарстан':'Республика Татарстан (Татарстан)', 'тыва':'Республика Тыва (Тува)',
'чечня':'Чеченская Республика', 'чеченская':'Чеченская Республика',
'чуваш':'Чувашская Республика — Чувашия', 'кемеров':'Кемеровская область — Кузбасс',
'владимир':'Владимирская область', 'воронежская область':'Воронежская область',
'Bолгоградская область':'Волгоградская область',
'москва':'г. Москва', 'санкт':'г. Санкт-Петербург', 'севастополь':'г. Севастополь',
'еврейская':'Еврейская автономная область', 'ямало-ненецкий':'Ямало-Ненецкий автономный округ',
'ханты-мансийский':'Ханты-Мансийский автономный округ — Югра',
'ненецкий автономный округ (архангельская область)':'Ненецкий автономный округ',
'ненецкий авт. округ':'Ненецкий автономный округ', 'архангельская':'Архангельская область', 'чукотский авт. округ':'Чукотский автономный округ',
'тюменская':'Тюменская область', '\nненецкий':'Ненецкий автономный округ', 'ханты-мансийский ао':'Ханты-Мансийский автономный округ — Югра',
'ямало-ненецкий ао':'Ямало-Ненецкий автономный округ', 'тюменская область без автономного округа':'Тюменская область',
'ямало-ненецкий автономный округ':'Ямало-Ненецкий автономный округ'}

Cоздадим функцию обработки таблиц

In [27]:
# Cоздадим функцию смены названия региона
def change_name(name):
    for n in names:
        if n in name.lower():
            return names[n]
    return name
# Функцию обработки таблиц
def prepare_table(Table):
    # Переименуем первый столбец
    df = Table.rename(columns={Table.columns[0]: "Регион"})
    # Выявим строки содержащие федеральный округ, федерация и район и исключим их
    fo = df[df['Регион'].str.contains('федеральный округ|Федерация|район|числе')].index
    df.drop(fo, axis = 0, inplace=True)
    # Унифицируем названия регионов
    df['Регион'] = df['Регион'].apply(change_name)
    # Назначим название региона индексом
    df["Регион"] = df["Регион"].str.strip()
    df.set_index('Регион', inplace=True)
    return df
# Удалим дублирующиеся индексы 
def Dupl_index(df):  
    dup_idx = df.index.duplicated()
    df = df.loc[~dup_idx]
    return df

Среднедушевые денежные доходы (в месяц), руб

In [28]:
cash_income = pd.read_excel('social_russia_data\cash_real_income_wages_2015_2020.xlsx')
cash_income = prepare_table(cash_income)
cash_income = Dupl_index(cash_income)
print(cash_income.shape)
cash_income.head(3)

(85, 6)


,2015,2016,2017,2018,2019,2020
Регион,,,,,,
Белгородская область,28043.0,29799.0,30342.0,30778.0,32352.0,32841.0
Брянская область,23428.0,24006.0,25107.0,26585.0,28371.0,28596.0
Владимирская область,22712.0,22365.0,23554.0,23539.0,25358.0,25922.0


Число умерших на первом году жизни детей за год, по всем регионам, в сельской местности

In [29]:
children_rural = pd.read_excel('social_russia_data/child_mortality_rural_1990_2021.xls', skiprows=2)
children_rural = prepare_table(children_rural)
children_rural = Dupl_index(children_rural)
# Удалим лишние колонки Оба пола и Сельское население - везде
children_rural.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)
# Удалим столбцы до 2016 года
old_d = list(map(lambda x: str(x),range(1990,2016)))
children_rural.drop(old_d, axis = 1, inplace=True)
print(children_rural.shape)
children_rural.head(3)

(91, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,36.0,34.0,33.0,16.0,22.0,20.0
Брянская область,36.0,31.0,12.0,11.0,12.0,13.0
Владимирская область,28.0,17.0,22.0,15.0,10.0,14.0


Число умерших на первом году жизни детей за год, по всем регионам, в городской местности.


In [30]:
children_urban = pd.read_excel('social_russia_data/child_mortality_urban_1990_2021.xls', skiprows=2)
children_urban = prepare_table(children_urban)
children_urban = Dupl_index(children_urban)
# Удалим лишние колонки Оба пола и Сельское население - везде
children_urban.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)
# Удалим столбцы до 2016 года
old_d = list(map(lambda x: str(x),range(1990,2016)))
children_urban.drop(old_d, axis = 1, inplace=True)
print(children_urban.shape)
children_urban.head(3)

(91, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,72.0,43.0,40.0,23.0,25.0,34.0
Брянская область,67.0,65.0,33.0,26.0,29.0,14.0
Владимирская область,72.0,59.0,45.0,43.0,51.0,52.0


Число людей с инвалидностью по регионам, по месяцам, по возрастным группам.


In [31]:
disability_group = pd.read_csv('social_russia_data/disabled_total_by_age_2017_2022.csv')
disability_group = prepare_table(disability_group)
# Сгруппируем по дате и региону общее количество людей
disability_group['date'] = pd.to_datetime(disability_group['date']).dt.year
disability_group = disability_group.groupby(['Регион','date'])['total'].sum().unstack()
print(disability_group.shape)
disability_group.head(3)

(86, 6)


date,2017,2018,2019,2020,2021,2022
Регион,,,,,,
Алтайский край,1987350.0,2120203.0,2100077.0,2060817.0,1986766.0,646296.0
Амурская область,816315.0,797050.0,785693.0,766768.0,733732.0,236444.0
Архангельская область,1056888.0,1038542.0,1031766.0,1016584.0,978993.0,316492.0


Валовой региональный продукт на душу населения, в рублях.

In [32]:
grp_df = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', skiprows=2)
grp_df = prepare_table(grp_df)
grp_df = Dupl_index(grp_df)
# Удалим столбцы до 2016 года
old_d = list(map(lambda x: str(x),range(1996,2016)))
grp_df.drop(old_d, axis = 1, inplace=True)
print(grp_df.shape)
grp_df.head(3)

(85, 5)


,2016,2017,2018,2019,2020
Регион,,,,,
Белгородская область,501467.8,539720.5,588641.5,617426.5,646569.0
Брянская область,258752.7,280630.1,304547.1,332442.8,347204.5
Владимирская область,309713.7,325043.1,349856.4,394560.3,410443.6


Размер жилой площади в расчете на члена домохозяйства за 2020г , м2

In [33]:
hous_df = pd.read_excel('social_russia_data/housing_2020.xlsx')
hous_df = prepare_table(hous_df)
hous_df = Dupl_index(hous_df)
square_liv = hous_df[['Размер жилой площади в расчете на члена домохозяйства']]
dif = list(set(square_liv.index).difference(set(grp_df.index)))
square_liv.drop(dif, axis = 0, inplace=True)
print(hous_df.shape)
hous_df.head(3)

(85, 8)


,"Число домохозяйств, проживающих во всех типах жилых помещений","в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство
Регион,,,,,,,,
Белгородская область,100.0,86.3,11.7,2.0,0.0,25.1,17.8,2.6
Брянская область,100.0,94.7,4.6,0.6,0.0,30.5,24.1,3.2
Bладимирская область,100.0,81.8,15.8,2.5,0.0,24.2,16.1,2.2


Заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний

In [34]:
morbidity_groups = pd.read_excel('social_russia_data/morbidity_2005_2020_age_disease.xls', skiprows=2)
morbidity_groups = prepare_table(morbidity_groups)
morbidity_groups.rename(columns={'Unnamed: 1':"Заболевания",'Unnamed: 2':"Группа"}, inplace=True)
morbidity_groups = morbidity_groups[morbidity_groups['Группа'] == 'Всего']
# Выделим датафрейм с общим показателем заболеваемости на 100тыс. человек населения за 2016г(только он попадает в выборку)
mor_all_df = morbidity_groups[morbidity_groups['Заболевания'].isin(['Все заболевания','Болезни системы кровообращения',\
    'Болезни органов дыхания','Болезни эндокринной системы, расстройства питания и нарушения обмена веществ'])].loc[:,['2016','Заболевания']]
mor_all_df = mor_all_df.dropna(axis=0)
mor_all_df = mor_all_df.groupby(['Регион','Заболевания'])['2016'].sum().unstack()
mor_all_df.rename(columns={'Все заболевания':'Заболеваемость на 100 тыс человек населения, чел.'}, inplace=True)

print(mor_all_df.shape)
mor_all_df.head(3)

(85, 4)


Заболевания,Болезни органов дыхания,Болезни системы кровообращения,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Заболеваемость на 100 тыс человек населения, чел."
Регион,,,,
Алтайский край,40416.4,5383.2,3885.0,112764.7
Амурская область,35518.0,2132.2,1147.3,83412.0
Архангельская область,44563.5,2452.0,1350.8,99842.1


Процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам.

In [35]:
people_poverty = pd.read_csv('social_russia_data/poverty_percent_by_regions_1992_2020.csv')
people_poverty = prepare_table(people_poverty)
# Перегруппируем  в датафрейм и удалим лишние столбцы
people_poverty.rename(columns={'year':"Года"}, inplace=True)
people_poverty = people_poverty.groupby(['Регион','Года'])['poverty_percent'].sum().unstack()
people_poverty.drop(people_poverty.columns[0:21],axis=1,inplace=True)

print(people_poverty.shape)
people_poverty.head(3)

(85, 5)


Года,2016,2017,2018,2019,2020
Регион,,,,,
Алтайский край,17.8,17.5,17.4,17.6,17.5
Амурская область,17.0,16.7,15.6,15.7,15.2
Архангельская область,30.1,28.2,26.0,26.3,25.1


Численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг.


In [36]:
population = pd.read_excel(
        'social_russia_data\population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)
population = prepare_table(population)
population = Dupl_index(population)
# Выявим строки содержащие в себе лишние регионы и исключим их
old_d = list(set(population.index).difference(set(people_poverty.index)))
population.drop(old_d, axis = 0, inplace=True)
# Удалим строки в названии столбцов
newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

# Изменим тип столбцов с int32 на object
old_column_name = [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
new_column_name = ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

for i in range(len(old_column_name)):
    population = population.rename(
    columns={old_column_name[i]: new_column_name[i]})

print(population.shape)
population.head(3)

(85, 24)


,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Регион,,,,,,,,,,,,,,,,,,,,,
Белгородская область,1494868.0,1501699.0,1506976.0,1508137.0,1511899.0,1513860.0,1511662.0,1511715.0,1514153.0,1520102.0,...,1540985.0,1544108.0,1547936.0,1550137.0,1552865.0,1549876.0,1547418.0,1549151.0,1541259.0,1531917.0
Брянская область,1437471.0,1423178.0,1407965.0,1391430.0,1375004.0,1360249.0,1344132.0,1327652.0,1312748.0,1303343.0,...,1253666.0,1242599.0,1232940.0,1225741.0,1220530.0,1210982.0,1200187.0,1192491.0,1182682.0,1168771.0
Владимирская область,1592184.0,1575507.0,1558052.0,1539179.0,1520057.0,1509571.0,1497598.0,1486453.0,1475861.0,1466743.0,...,1421742.0,1413321.0,1405613.0,1397168.0,1389599.0,1378337.0,1365805.0,1358416.0,1342099.0,1323659.0


Расходы на социальную политику от общих расходов бюджета региона, % в год.

In [37]:
Social_expenses = pd.read_excel('social_russia_data/welfare_expense_share_2015_2020.xlsx')
Social_expenses = prepare_table(Social_expenses)
print(Social_expenses.shape)
Social_expenses.head(3)

(85, 6)


,2015,2016,2017,2018,2019,2020
Регион,,,,,,
Белгородская область,11.3,11.8,16.4,15.6,13.8,16.1
Брянская область,22.0,26.6,24.7,23.7,20.7,23.1
Владимирская область,18.1,18.0,24.0,23.4,23.4,26.0


Оборот розничной торговли на душу населения, в рублях.

In [38]:
Retail_turnover = pd.read_excel('social_russia_data\etail_turnover_per_capita_2000_2021.xls', skiprows=2)
# Удалим ненужные столбцы и строки
Retail_turnover.drop([0], axis=0, inplace=True)
Retail_turnover.drop(Retail_turnover.columns[1:18], axis=1,inplace=True)
Retail_turnover = prepare_table(Retail_turnover)
Retail_turnover = Dupl_index(Retail_turnover)
old_d = list(set(Retail_turnover.index).difference(set(people_poverty.index)))
Retail_turnover.drop(old_d, axis = 0, inplace=True)
print(Retail_turnover.shape)
Retail_turnover.head(3)

(85, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,192504,203298,217059,230704,237076,259151
Брянская область,179767,192775,209987,225818,225643,257679
Владимирская область,143643,153088,164519,175251,177680,207670


Распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам.

In [39]:
demographic_groups_17_df = pd.read_excel('social_russia_data/poverty_socdem_2017.xls',skiprows=2)
demographic_groups_17_df = prepare_table(demographic_groups_17_df)
demographic_groups_17_df = Dupl_index(demographic_groups_17_df)
demographic_groups_17_df = demographic_groups_17_df.fillna(0)
demographic_groups_18_df = pd.read_excel('social_russia_data/poverty_socdem_2018.xls',skiprows=2)
demographic_groups_18_df = prepare_table(demographic_groups_18_df)
demographic_groups_18_df = Dupl_index(demographic_groups_18_df)
demographic_groups_18_df = demographic_groups_18_df.fillna(0)
demographic_groups_19_df = pd.read_excel('social_russia_data/poverty_socdem_2019.xls',skiprows=2)
demographic_groups_19_df = prepare_table(demographic_groups_19_df)
demographic_groups_19_df = Dupl_index(demographic_groups_19_df)
demographic_groups_19_df = demographic_groups_19_df.fillna(0)
demographic_groups_20_df = pd.read_excel('social_russia_data/poverty_socdem_2020.xls',skiprows=2)
demographic_groups_20_df = prepare_table(demographic_groups_20_df)
demographic_groups_20_df = Dupl_index(demographic_groups_20_df)
demographic_groups_20_df = demographic_groups_20_df.fillna(0)

print(demographic_groups_17_df.shape)
demographic_groups_17_df.head(3)

(85, 4)


,Все население,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста
Регион,,,,
Белгородская область,100,43.4,11.8,44.8
Брянская область,100,42.9,4.9,52.2
Владимирская область,100,34.8,8.6,56.6


Создадим признак % малоимущих детей

In [40]:
ps_17_deti = demographic_groups_17_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_18_deti = demographic_groups_18_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_19_deti = demographic_groups_19_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_20_deti = demographic_groups_20_df.loc[:, ['Дети в возрасте до 16 лет']]

ps_17_deti.rename(columns={'Дети в возрасте до 16 лет':'2017'},inplace=True)
ps_18_deti.rename(columns={'Дети в возрасте до 16 лет':'2018'},inplace=True)
ps_19_deti.rename(columns={'Дети в возрасте до 16 лет':'2019'},inplace=True)
ps_20_deti.rename(columns={'Дети в возрасте до 16 лет':'2020'},inplace=True)

ps_deti = ps_17_deti.merge(ps_18_deti,on=['Регион'],how='left')
ps_deti = ps_deti.merge(ps_19_deti,on=['Регион'],how='left')
ps_deti = ps_deti.merge(ps_20_deti,on=['Регион'],how='left')
ps_deti = Dupl_index(ps_deti)
print(ps_deti.shape)
ps_deti.head(3)

(85, 4)


,2017,2018,2019,2020
Регион,,,,
Белгородская область,43.4,44.8,0.0,0.0
Брянская область,42.9,40.3,37.5,44.8
Владимирская область,34.8,48.9,34.8,0.0


Создадим признак % малоимущих пенсионеров

In [41]:
ps_17_pens = demographic_groups_17_df.loc[:, ['Население старше трудоспособного возраста']]
ps_18_pens = demographic_groups_18_df.loc[:, ['Население старше трудоспособного возраста']]
ps_19_pens = demographic_groups_19_df.loc[:, ['Население старше трудоспособного возраста']]
ps_20_pens = demographic_groups_20_df.loc[:, ['Население старше трудоспособного возраста']]

ps_17_pens.rename(columns={'Население старше трудоспособного возраста':'2017'},inplace=True)
ps_18_pens.rename(columns={'Население старше трудоспособного возраста':'2018'},inplace=True)
ps_19_pens.rename(columns={'Население старше трудоспособного возраста':'2019'},inplace=True)
ps_20_pens.rename(columns={'Население старше трудоспособного возраста':'2020'},inplace=True)

ps_pens = ps_17_pens.merge(ps_18_pens,on=['Регион'],how='left')
ps_pens = ps_pens.merge(ps_19_pens,on=['Регион'],how='left')
ps_pens = ps_pens.merge(ps_20_pens,on=['Регион'],how='left')
ps_pens = Dupl_index(ps_pens)
print(ps_pens.shape)
ps_pens.head(3)

(85, 4)


,2017,2018,2019,2020
Регион,,,,
Белгородская область,11.8,12.9,0.0,0.0
Брянская область,4.9,1.8,5.1,3.7
Владимирская область,8.6,8.3,4.8,0.0


Создадим признак % малоимущих трудоспособных

In [42]:
ps_17_trud = demographic_groups_17_df.loc[:, ['Население трудоспособного возраста']]
ps_18_trud = demographic_groups_18_df.loc[:, ['Население трудоспособного возраста']]
ps_19_trud = demographic_groups_19_df.loc[:, ['Население трудоспособного возраста']]
ps_20_trud = demographic_groups_20_df.loc[:, ['Население трудоспособного возраста']]

ps_17_trud.rename(columns={'Население трудоспособного возраста':'2017'},inplace=True)
ps_18_trud.rename(columns={'Население трудоспособного возраста':'2018'},inplace=True)
ps_19_trud.rename(columns={'Население трудоспособного возраста':'2019'},inplace=True)
ps_20_trud.rename(columns={'Население трудоспособного возраста':'2020'},inplace=True)

ps_trud = ps_17_trud.merge(ps_18_trud,on=['Регион'],how='left')
ps_trud = ps_trud.merge(ps_19_trud,on=['Регион'],how='left')
ps_trud = ps_trud.merge(ps_20_trud,on=['Регион'],how='left')
ps_trud = Dupl_index(ps_trud)
print(ps_trud.shape)
ps_trud.head(3)

(85, 4)


,2017,2018,2019,2020
Регион,,,,
Белгородская область,44.8,42.3,0.0,0.0
Брянская область,52.2,57.9,57.4,51.6
Владимирская область,56.6,42.8,60.4,0.0


Объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности за 2005–2016 гг., 2017–2020 гг. (в тысячах рублей, значение показателя за год, полный круг)

In [43]:
Scope_work = pd.read_csv('social_russia_data\egional_production_2017_2020.csv')
Scope_work = prepare_table(Scope_work)

# Выделим датафрейм с показателем по добыче полезных ископаемых
rep_mining_df = Scope_work[Scope_work['production_field'] == 'ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по обрабатывающим производствам
rep_manu_df = Scope_work[Scope_work['production_field'] == 'ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по промышленности
rep_indust_df = Scope_work[Scope_work['production_field'] == 'Промышленное производство (промышленность)'].loc[:,'2017':'2020']
# Выделим датафрейм с показателем по обеспечения электроэнергией
rep_electr_df = Scope_work[Scope_work['production_field'] == 'ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА'].loc[:,'2017':'2020']

#Переведем значения в руб
rep_electr_df *1000
rep_manu_df *1000
rep_indust_df *1000
rep_mining_df *1000
print(rep_mining_df.shape)
rep_mining_df.head(3)

(85, 4)


,2017,2018,2019,2020
Регион,,,,
Белгородская область,116588313.5,148863422.7,178389894.9,189803813.7
Брянская область,277427.7,261549.5,338027.3,310665.9
Владимирская область,4003911.4,5005218.9,5425671.9,4965110.5


Количество безработных в России по регионам. Данные с сайта https://gogov.ru

In [44]:
unemployed = pd.read_csv('social_russia_data\ezrabotnye.csv')
unemployed = prepare_table(unemployed)
print(unemployed.shape)
unemployed.head(3)

(85, 21)


,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Регион,,,,,,,,,,,,,,,,,,,,,
Республика Адыгея (Адыгея),27900.0,27000.0,29100.0,32900.0,26200.0,28000.0,21400.0,14700.0,15200.0,19100.0,...,16500.0,16600.0,17300.0,17700.0,18200.0,17600.0,17300.0,16400.0,17000.0,16800.0
Алтай,8800.0,10400.0,9300.0,9300.0,8900.0,10900.0,8800.0,12300.0,13700.0,12300.0,...,11500.0,11500.0,10200.0,9700.0,11900.0,11800.0,10800.0,10500.0,13800.0,11600.0
Алтайский край,123700.0,106100.0,155300.0,117600.0,112900.0,114400.0,81700.0,105100.0,152000.0,110600.0,...,74400.0,95800.0,83400.0,94800.0,99600.0,78200.0,70300.0,66100.0,65200.0,60600.0


Рождённые в этом месяце, по регионам, без учёта мертворождённых.

In [45]:
borned_df = pd.read_csv('social_russia_data\wborn_2006_2022_monthly.csv',sep=';', decimal=',')
borned_df = prepare_table(borned_df)
borned_df = Dupl_index(borned_df)
# Выявим строки содержащие в себе лишние регионы и исключим их
old_d = list(set(borned_df.index).difference(set(people_poverty.index)))
borned_df.drop(old_d, axis = 0, inplace=True)
borned_df = borned_df.drop(columns=borned_df.iloc[:, -6:])

# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in borned_df.columns:
        if str(year) in str(col):
            cols.append(col)
    borned_df[year] = borned_df[cols].sum(axis=1)
    borned_df.drop(columns=cols, inplace=True)
borned_df.drop(columns=2022, inplace=True)  # Неполный год
borned_df = borned_df.round(0).astype(int)
borned_df.columns = pd.to_datetime(borned_df.columns, format='%Y').year

# Изменим тип столбцов с int32 на object
old_column_name = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021]
new_column_name = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
                   '2015', '2016', '2017', '2018', '2019', '2020', '2021']

for i in range(len(old_column_name)):
    borned_df = borned_df.rename(
    columns={old_column_name[i]: new_column_name[i]})

print(borned_df.shape)
borned_df.head(3)

(85, 16)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Регион,,,,,,,,,,,,,,,,
Белгородская область,14269,15666,16810,16911,16669,16894,17913,17883,17822,17864,17261,15113,14318,13198,12352,12223
Брянская область,12042,13283,14319,14406,13773,13806,14376,13855,13658,14067,13379,11553,11124,10028,9480,9047
Владимирская область,13915,14679,15699,15520,15540,15644,16445,15777,15817,16284,15662,13395,12658,11158,10120,9780


Заболеваемость с впервые в жизни установленным диагнозом алкоголизма и наркомания на 100 тыс. населения

In [46]:
def alco_readout(file, sheet):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    df = prepare_table(df)
    return df


file = 'social_russia_data\drug_alco.xlsx'
alco = alco_readout(file, 0)
alco1718 = alco_readout(file, 1)
drug = alco_readout(file, 2)
drug1718 = alco_readout(file, 3)
alco = pd.concat([alco, alco1718], axis=1).sort_index()
alco = Dupl_index(alco)
drug = pd.concat([drug, drug1718], axis=1).sort_index()
drug = Dupl_index(drug)

print(alco.shape)
display('Алкоголизм', alco.head(3))
print(drug.shape)
display('Наркомания', drug.head(3))

(87, 14)


'Алкоголизм'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Регион,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7


(87, 14)


'Наркомания'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Регион,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2


# Преобразование показателей и объединение в единый датафрейм

In [47]:
# Среднегодовая численность населения, чел
pop_mean = population.mean(axis=1)
# Среднедушевые денежные доходы, руб
cash_income_mean = cash_income.loc[:,2016:2020].mean(axis=1)
# Среднегодовая детская смертность на душу населения, чел
children_total = children_rural + children_urban 
children_total = children_total / population
children_total_mean = children_total.loc[:,'2016':'2020'].mean(axis=1)
# Среднегодовое число людей инвалидов , чел
disability_group = disability_group.loc[:,2017:2020].mean(axis=1)
disability_group_mean = disability_group / pop_mean
# Среднегодовой валовой региональный продукт на душу населения, руб
grp_mean = grp_df.mean(axis=1)
# Среднегодовой процент людей, живущих за чертой бедности
people_poverty_mean = people_poverty.mean(axis=1)
# Среднегодовой процент расходов на социальную политику
Social_expenses_mean = Social_expenses.loc[:,2016.0:2020.0].mean(axis=1)
# Средегодовой оборот розничной торговли на душу населения, руб
Retail_turnover_mean = Retail_turnover.loc[:,'2016':'2020'].mean(axis=1)

# Средегодовой % малоимущих детей, доля
ps_deti_mean = ps_deti.mean(axis=1)
# Средегодовой % малоимущих пенсионеров, доля
ps_pens_mean = ps_pens.mean(axis=1)
# Средегодовой % малоимущих трудоспособных людей, доля
ps_trud_mean = ps_trud.mean(axis=1)
# Средегодовой объем производимой добычи полезных ископаемых, руб на душу населения
rep_mining_per = rep_mining_df / population.loc[:,'2017':'2020']
rep_mining_mean = rep_mining_per.mean(axis=1).fillna(0)
rep_mining_mean = Dupl_index(rep_mining_mean)
# Средегодовой объем производимой промышленности, руб на душу населения
rep_indust_per = rep_indust_df / population.loc[:,'2017':'2020']
rep_indust_mean = rep_indust_per.mean(axis=1).fillna(0)
rep_indust_mean = Dupl_index(rep_indust_mean)
# Средегодовой объем производимого обрабатывающего производства, руб на душу населения
rep_manu_per = rep_manu_df / population.loc[:,'2017':'2020']
rep_manu_mean = rep_manu_per.mean(axis=1).fillna(0)
rep_manu_mean = Dupl_index(rep_manu_mean)
# Средегодовой объем производимой электрической энергии, газом и паром, руб на душу населения
rep_electr_per = rep_electr_df / population.loc[:,'2017':'2020']
rep_electr_mean = rep_electr_per.mean(axis=1).fillna(0)
rep_electr_mean = Dupl_index(rep_electr_mean)
# Среднегодовая численность безработных, чел 
unemp = unemployed.loc[:,'2016':'2020'].mean(axis=1)
unemployed_mean = unemp / pop_mean
unemployed_mean = Dupl_index(unemployed_mean)
# Среднегодовая рождаемость, чел 
borned_per = borned_df / population
borned_mean = borned_per.mean(axis=1)

alco = alco.replace ( r'-' , np.nan , regex= True )
drug = drug.replace ( r'-' , np.nan , regex= True )
alco[2016] =  alco[2016].fillna(alco[2016].median())
alco[2017] =  alco[2017].fillna(alco[2017].median())
alco[2018] =  alco[2018].fillna(alco[2018].median())
drug[2016] =  drug[2016].fillna(drug[2016].median())
drug[2017] =  drug[2017].fillna(drug[2017].median())
drug[2018] =  drug[2018].fillna(drug[2018].median())
# Среднегодовая заболеваемость впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза на 100 тыс.  населения, чел 
alco_mean = alco.loc[:,2016:2018].mean(axis=1)
# Среднегодовая заболеваемость впервые в жизни установленным диагнозом наркомании на 100 тыс.  населения, чел 
drug_mean = drug.loc[:,2016:2018].mean(axis=1)

Объединим все признаки в итоговом датасете 

In [48]:
Result_regions = pd.concat([pop_mean, cash_income_mean, children_total_mean, disability_group_mean, grp_mean, people_poverty_mean,
    Social_expenses_mean, Retail_turnover_mean, ps_deti_mean, ps_pens_mean, ps_trud_mean, rep_mining_mean, rep_indust_mean,
    rep_manu_mean, rep_electr_mean, unemployed_mean, borned_mean,drug_mean,alco_mean,mor_all_df,square_liv],axis=1).fillna(0)
Result_regions.rename(columns={0:'population', 1:'cash_income', 2:'child_mort', 3:'disabled', 4:'grp', 5:'poverty_%', 
    6:'Social_expenses_%', 7:'retail', 8:'poverty_deti_%', 9:'poverty_pens_%', 10:'poverty_trud_%', 11:'rep_mining', 
    12:'rep_industry', 13:'rep_manufacture', 14:'rep_electro', 15:'unemployed', 16:'borned', 17:'drug', 
    18:'alco','Болезни органов дыхания':'d_resp','Болезни системы кровообращения':'d_blood', 
    'Болезни эндокринной системы, расстройства питания и нарушения обмена веществ':'d_endo',
    'Заболеваемость на 100 тыс человек населения, чел.':'d_all',
    'Размер жилой площади в расчете на члена домохозяйства':'square_liv'})
Result_regions.columns=['population','cash_income','child_mort','disabled','grp','poverty_%','Social_expenses_%',
        'retail','poverty_deti_%', 'poverty_pens_%','poverty_trud_%','rep_mining','rep_industry','rep_manufacture',
        'rep_electro','unemployed', 'borned','drug','alco','d_resp','d_blood','d_endo','d_all','square_liv']
Result_regions = Result_regions.iloc[0:85]

Result_regions.to_csv('social_russia_data/Result_regions.csv')
print(Result_regions.shape)
Result_regions.head(3)

(85, 24)


,population,cash_income,child_mort,disabled,grp,poverty_%,Social_expenses_%,retail,poverty_deti_%,poverty_pens_%,...,rep_electro,unemployed,borned,drug,alco,d_resp,d_blood,d_endo,d_all,square_liv
Регион,,,,,,,,,,,,,,,,,,,,,
Белгородская область,1.528216e+06,31222.4,0.000044,1.628376,578765.06,7.68,14.74,216128.2,22.050,6.175,...,17.286679,0.000000,0.010295,2.366667,44.200000,27195.7,3995.8,813.6,74637.4,17.8
Брянская область,1.288930e+06,26533.0,0.000053,1.001405,304715.44,13.92,23.76,206798.0,41.375,3.875,...,14.387151,0.000000,0.010074,10.000000,94.100000,33294.4,3201.2,1641.5,77439.0,24.1
Владимирская область,1.449906e+06,24147.6,0.000052,1.044558,357923.42,13.20,22.96,162836.2,29.625,5.425,...,26.355518,0.024677,0.010033,10.466667,69.166667,46461.1,2715.0,1265.2,94163.6,0.0


# Вывод

Проведено считывание экономических и демографических данных из предоставленных файлов различного формата и из дополнительных источников. Результаты сохраняются в датасеты со стандартизованными названиями федеральных субъектов РФ (85 субъектов). На основе полученных датасетов сформирована финальная таблица, которая используется далее в проекте.